# Patient Follow-Up Letter Generator

This notebook demonstrates the process of creating a patient follow-up letter generator. The application extracts relevant information from uploaded transcript files and uses AI (GPT-4) to generate a detailed patient letter. The generated letter can be formatted and downloaded for further use.

## Workflow:
1. File Upload
2. Text Extraction
3. API Interaction for Letter Generation
4. Letter Formatting
5. Download the Letter


In [5]:
# Import necessary modules
import os
from PyPDF2 import PdfReader
from docx import Document

class FileManager:
    @staticmethod
    def save_uploaded_file(uploaded_file, folder="uploads"):
        """
        Saves the uploaded file to the specified folder and returns the file path.
        """
        os.makedirs(folder, exist_ok=True)
        file_path = os.path.join(folder, uploaded_file.name)
        with open(file_path, "wb") as f:
            f.write(uploaded_file.getbuffer())
        return file_path

    @staticmethod
    def extract_text_from_file(file_path):
        """
        Extracts and returns the text content from a file based on its type.
        Supports PDF, DOCX, and TXT formats.
        """
        if file_path.endswith('.pdf'):
            with open(file_path, 'rb') as f:
                reader = PdfReader(f)
                return "".join([page.extract_text() for page in reader.pages])
        elif file_path.endswith('.docx'):
            doc = Document(file_path)
            return "\n".join([para.text for para in doc.paragraphs])
        else:
            with open(file_path, 'r') as file:
                return file.read()

# Display file upload and extraction example
uploaded_file_path = "transcript1.txt"  # Replace with your file path
file_path = uploaded_file_path  # Directly use the file path
transcript = FileManager.extract_text_from_file(file_path)
print(transcript[:500])  # Display first 500 characters to confirm extraction
  # Display first 500 characters to confirm extraction


So, Mr. Thompson, thanks for coming in today. What brings you in to see me? Well, Iâ€™ve been having some sensitivity in my lower left teeth, especially when drinking cold water. Also, I noticed some pain when chewing on that side. I see. And how long have you been experiencing this discomfort? It started maybe two or three weeks ago. At first, it was just occasional sensitivity, but now it's pretty consistent, especially with anything cold or crunchy. Okay. And any other symptoms, like swelling


## General Work flow 
### tone - set- casual

In [9]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

client = OpenAI(api_key=os.environ.get('open_api_key'))

temperature = 0.3
max_tokens = 1000
top_p = 0.9

class ApiInteraction:
    def __init__(self):
        self.api_key = os.environ.get('open_api_key')
        if not self.api_key:
            raise ValueError("API key is missing from the environment variables")
        
        # Set the OpenAI API key
        client.api_key = self.api_key

    def generate_patient_letter(self, transcript, tone="empathetic"):
        """
        Generate a detailed and structured patient follow-up letter from a consultation transcript.
        The letter should be accurate, comprehensive, and easy for patients to understand, with tone variation.
        """

        tone_styles = {
            "empathetic": "Use a gentle, caring, and supportive tone, reassuring the patient and expressing empathy for their concerns.",
            "formal": "Maintain a polished, professional tone, focusing on clinical accuracy and straightforward explanations.",
            "casual": "Adopt a friendly, conversational tone to make the patient feel comfortable and at ease.",
        }

        selected_tone = tone_styles.get(tone.lower(), tone_styles["casual"])

        system_message = f"""
        You are an expert dentist and medical writer. Your task is to draft a personalized, patient-friendly follow-up letter based on the provided consultation transcript. 
        Carefully extract and organize relevant details, ensuring completeness and accuracy. Match the letter’s tone to the user's request.
        
        Tone: {selected_tone}
        
        Guidelines:
        - Extract doctor or physician's name and the patient's name and use it in the place holder 
        - If the the name or details of the doctor or patient are missing then omit it or leave the space.
        - If the date is given then use in th eplace holde or else place it as on you last visit 
        - Extract names, dates, symptoms, and observations from the transcript.
        - Summarize findings and treatments concisely but comprehensively.
        - Avoid section titles or headings to match real-world dental letters.
        - Fill in missing details wisely with general advice or omit sections if unavailable.
        - Ensure a smooth, natural flow with clear transitions.
        """

        user_message = f"""
        Dear [Patient Name],

        Thank you for visiting our dental practice on [Date of Appointment]. It was a pleasure meeting you and addressing your dental health concerns.

        Thank you for sharing your symptoms with us. Based on your consultation, we noted the following: 
        Main symptoms: [List main symptoms or omit if not mentioned]. Clinical observations revealed [describe clinical findings, including any inflammation, decay, plaque, or bone loss, or omit if not mentioned]. Relevant X-ray findings include [summarize results or omit if not available].

        Regarding your treatment, we recommend the following procedures: [List recommended treatments, e.g., filling replacement, deep cleaning, root canal, etc., or suggest common preventive measures]. We also suggest maintaining good oral hygiene through [specific hygiene advice or general tips like brushing twice daily and flossing]. Lifestyle adjustments, such as [reducing smoking or modifying diet, or skip if not discussed], may further improve your oral health.

        For your next steps, your scheduled appointment is on [next appointment date, or recommend scheduling one if missing]. In the meantime, we advise [short-term advice, like using desensitizing toothpaste, or provide general comfort measures].

        Please remember that your oral health is our top priority. If you have any questions or experience worsening symptoms, don’t hesitate to contact our office.

        Best wishes for your continued dental health,

        [Physician's Name]
        Physician - Chairsyde 
        ---
        
        Transcript:
        {transcript}
        
        Fill in the structure with extracted information, ensuring coherence and a natural flow.
        """

        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
        )

        return completion.choices[0].message.content

# Example call to API
api_interaction = ApiInteraction()
letter = api_interaction.generate_patient_letter(transcript)
print(letter)  # Display first 500 characters of generated letter


Dear Mr. Thompson,

Thank you for visiting our dental practice on your last visit. It was a pleasure meeting you and discussing your dental health concerns.

I appreciate you sharing your symptoms with us. Based on our consultation, you mentioned experiencing sensitivity in your lower left teeth, particularly when drinking cold water, as well as pain while chewing on that side. You noted that this discomfort started about two to three weeks ago and has become more consistent. Additionally, you reported some bleeding when brushing your teeth, mainly near your molars.

During the examination, I observed some plaque buildup along your lower molars and mild gum inflammation. Your gum probing scores indicated some inflammation, and I noticed some recession on your lower left side, which has exposed part of the tooth root. There is also an old composite filling on your lower left first molar, which you mentioned was placed about five or six years ago. When I tapped on your lower left tooth, 

In [15]:
class LetterFormatter:
    @staticmethod
    def format_letter(letter):
        """
        Format the generated letter to ensure proper structure and style.
        """
        clinic_header = """
                                Chairsyde Dental Clinic
                                Your Smile, Our Priority!\n"""
        
        clinic_footer = """
                        Thank you for trusting Chairsyde!
        """
        
        formatted_letter = clinic_header + letter + clinic_footer
        formatted_letter = formatted_letter.replace("\n", "\n")
        
        return formatted_letter

# Apply formatting to the generated letter
formatted_letter = LetterFormatter.format_letter(letter)
print(formatted_letter)  # Display first 500 characters of formatted letter



                                Chairsyde Dental Clinic
                                Your Smile, Our Priority!
Dear Mr. Thompson,

Thank you for visiting our dental practice on your last visit. It was a pleasure meeting you and discussing your dental health concerns.

I appreciate you sharing your symptoms with us. Based on our consultation, you mentioned experiencing sensitivity in your lower left teeth, particularly when drinking cold water, as well as pain while chewing on that side. You noted that this discomfort started about two to three weeks ago and has become more consistent. Additionally, you reported some bleeding when brushing your teeth, mainly near your molars.

During the examination, I observed some plaque buildup along your lower molars and mild gum inflammation. Your gum probing scores indicated some inflammation, and I noticed some recession on your lower left side, which has exposed part of the tooth root. There is also an old composite filling on your lower lef

## Tone changes 
## Tone - casual

In [16]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

client = OpenAI(api_key=os.environ.get('open_api_key'))

temperature = 0.3
max_tokens = 1000
top_p = 0.9

class ApiInteraction:
    def __init__(self):
        self.api_key = os.environ.get('open_api_key')
        if not self.api_key:
            raise ValueError("API key is missing from the environment variables")
        
        # Set the OpenAI API key
        client.api_key = self.api_key

    def generate_patient_letter(self, transcript, tone="empathetic"):
        """
        Generate a detailed and structured patient follow-up letter from a consultation transcript.
        The letter should be accurate, comprehensive, and easy for patients to understand, with tone variation.
        """

        tone_styles = {
            "empathetic": "Use a gentle, caring, and supportive tone, reassuring the patient and expressing empathy for their concerns.",
            "formal": "Maintain a polished, professional tone, focusing on clinical accuracy and straightforward explanations.",
            "casual": "Adopt a friendly, conversational tone to make the patient feel comfortable and at ease.",
        }

        selected_tone = tone_styles.get(tone.lower(), tone_styles["casual"])

        system_message = f"""
        You are an expert dentist and medical writer. Your task is to draft a personalized, patient-friendly follow-up letter based on the provided consultation transcript. 
        Carefully extract and organize relevant details, ensuring completeness and accuracy. Match the letter’s tone to the user's request.
        
        Tone: {selected_tone}
        
        Guidelines:
        - Extract doctor or physician's name and the patient's name and use it in the place holder 
        - If the the name or details of the doctor or patient are missing then omit it or leave the space.
        - If the date is given then use in th eplace holde or else place it as on you last visit 
        - Extract names, dates, symptoms, and observations from the transcript.
        - Summarize findings and treatments concisely but comprehensively.
        - Avoid section titles or headings to match real-world dental letters.
        - Fill in missing details wisely with general advice or omit sections if unavailable.
        - Ensure a smooth, natural flow with clear transitions.
        """

        user_message = f"""
        Dear [Patient Name],

        Thank you for visiting our dental practice on [Date of Appointment]. It was a pleasure meeting you and addressing your dental health concerns.

        Thank you for sharing your symptoms with us. Based on your consultation, we noted the following: 
        Main symptoms: [List main symptoms or omit if not mentioned]. Clinical observations revealed [describe clinical findings, including any inflammation, decay, plaque, or bone loss, or omit if not mentioned]. Relevant X-ray findings include [summarize results or omit if not available].

        Regarding your treatment, we recommend the following procedures: [List recommended treatments, e.g., filling replacement, deep cleaning, root canal, etc., or suggest common preventive measures]. We also suggest maintaining good oral hygiene through [specific hygiene advice or general tips like brushing twice daily and flossing]. Lifestyle adjustments, such as [reducing smoking or modifying diet, or skip if not discussed], may further improve your oral health.

        For your next steps, your scheduled appointment is on [next appointment date, or recommend scheduling one if missing]. In the meantime, we advise [short-term advice, like using desensitizing toothpaste, or provide general comfort measures].

        Please remember that your oral health is our top priority. If you have any questions or experience worsening symptoms, don’t hesitate to contact our office.

        Best wishes for your continued dental health,

        [Physician's Name]
        Physician - Chairsyde 
        ---
        
        Transcript:
        {transcript}
        
        Fill in the structure with extracted information, ensuring coherence and a natural flow.
        """

        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
        )

        return completion.choices[0].message.content

# Example call to API
api_interaction = ApiInteraction()
letter = api_interaction.generate_patient_letter(transcript)
print(letter)  # Display first 500 characters of generated letter


Dear Mr. Thompson,

Thank you for visiting our dental practice on your last visit. It was a pleasure meeting you and discussing your dental health concerns.

I appreciate you sharing your symptoms with us. Based on our consultation, you mentioned experiencing sensitivity in your lower left teeth, particularly when drinking cold water, along with some pain while chewing on that side. You noted that this discomfort started about two to three weeks ago and has become more consistent. Additionally, you reported some bleeding when brushing your teeth, especially near your molars.

During the examination, I observed some plaque buildup along your lower molars and mild gum inflammation. There was also an old composite filling on your lower left first molar, which you mentioned was placed about five or six years ago. Your gum probing scores indicated some inflammation, and I noticed some recession on the lower left side, exposing part of the tooth root. When I tapped on your lower left molar, 

### Tone - empathetic

In [17]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

client = OpenAI(api_key=os.environ.get('open_api_key'))

temperature = 0.3
max_tokens = 1000
top_p = 0.9

class ApiInteraction:
    def __init__(self):
        self.api_key = os.environ.get('open_api_key')
        if not self.api_key:
            raise ValueError("API key is missing from the environment variables")
        
        # Set the OpenAI API key
        client.api_key = self.api_key

    def generate_patient_letter(self, transcript, tone="empathetic"):
        """
        Generate a detailed and structured patient follow-up letter from a consultation transcript.
        The letter should be accurate, comprehensive, and easy for patients to understand, with tone variation.
        """

        tone_styles = {
            "empathetic": "Use a gentle, caring, and supportive tone, reassuring the patient and expressing empathy for their concerns.",
            "formal": "Maintain a polished, professional tone, focusing on clinical accuracy and straightforward explanations.",
            "casual": "Adopt a friendly, conversational tone to make the patient feel comfortable and at ease.",
        }

        selected_tone = tone_styles.get(tone.lower(), tone_styles["empathetic"])

        system_message = f"""
        You are an expert dentist and medical writer. Your task is to draft a personalized, patient-friendly follow-up letter based on the provided consultation transcript. 
        Carefully extract and organize relevant details, ensuring completeness and accuracy. Match the letter’s tone to the user's request.
        
        Tone: {selected_tone}
        
        Guidelines:
        - Extract doctor or physician's name and the patient's name and use it in the place holder 
        - If the the name or details of the doctor or patient are missing then omit it or leave the space.
        - If the date is given then use in th eplace holde or else place it as on you last visit 
        - Extract names, dates, symptoms, and observations from the transcript.
        - Summarize findings and treatments concisely but comprehensively.
        - Avoid section titles or headings to match real-world dental letters.
        - Fill in missing details wisely with general advice or omit sections if unavailable.
        - Ensure a smooth, natural flow with clear transitions.
        """

        user_message = f"""
        Dear [Patient Name],

        Thank you for visiting our dental practice on [Date of Appointment]. It was a pleasure meeting you and addressing your dental health concerns.

        Thank you for sharing your symptoms with us. Based on your consultation, we noted the following: 
        Main symptoms: [List main symptoms or omit if not mentioned]. Clinical observations revealed [describe clinical findings, including any inflammation, decay, plaque, or bone loss, or omit if not mentioned]. Relevant X-ray findings include [summarize results or omit if not available].

        Regarding your treatment, we recommend the following procedures: [List recommended treatments, e.g., filling replacement, deep cleaning, root canal, etc., or suggest common preventive measures]. We also suggest maintaining good oral hygiene through [specific hygiene advice or general tips like brushing twice daily and flossing]. Lifestyle adjustments, such as [reducing smoking or modifying diet, or skip if not discussed], may further improve your oral health.

        For your next steps, your scheduled appointment is on [next appointment date, or recommend scheduling one if missing]. In the meantime, we advise [short-term advice, like using desensitizing toothpaste, or provide general comfort measures].

        Please remember that your oral health is our top priority. If you have any questions or experience worsening symptoms, don’t hesitate to contact our office.

        Best wishes for your continued dental health,

        [Physician's Name]
        Physician - Chairsyde 
        ---
        
        Transcript:
        {transcript}
        
        Fill in the structure with extracted information, ensuring coherence and a natural flow.
        """

        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
        )

        return completion.choices[0].message.content

# Example call to API
api_interaction = ApiInteraction()
letter = api_interaction.generate_patient_letter(transcript)
print(letter)  # Display first 500 characters of generated letter


Dear Mr. Thompson,

Thank you for visiting our dental practice on your last visit. It was a pleasure meeting you and discussing your dental health concerns.

I appreciate you sharing your symptoms with us. Based on our consultation, you have been experiencing sensitivity in your lower left teeth, particularly when drinking cold water, along with some pain while chewing on that side. You mentioned that this discomfort began about two to three weeks ago and has become more consistent, especially with cold or crunchy foods. Additionally, you noted some bleeding when brushing your teeth, primarily near your molars.

During the examination, I observed some plaque buildup along your lower molars and mild gum inflammation. Your gum health indicated some recession on the lower left side, exposing part of the tooth root. When I tapped on your lower left molar, you reported some pain, which is concerning. The X-rays revealed mild bone loss around your lower molars and an area of decay beneath yo

### Tone - formal

In [18]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

client = OpenAI(api_key=os.environ.get('open_api_key'))

temperature = 0.3
max_tokens = 1000
top_p = 0.9

class ApiInteraction:
    def __init__(self):
        self.api_key = os.environ.get('open_api_key')
        if not self.api_key:
            raise ValueError("API key is missing from the environment variables")
        
        # Set the OpenAI API key
        client.api_key = self.api_key

    def generate_patient_letter(self, transcript, tone="formal"):
        """
        Generate a detailed and structured patient follow-up letter from a consultation transcript.
        The letter should be accurate, comprehensive, and easy for patients to understand, with tone variation.
        """

        tone_styles = {
            "empathetic": "Use a gentle, caring, and supportive tone, reassuring the patient and expressing empathy for their concerns.",
            "formal": "Maintain a polished, professional tone, focusing on clinical accuracy and straightforward explanations.",
            "casual": "Adopt a friendly, conversational tone to make the patient feel comfortable and at ease.",
        }

        selected_tone = tone_styles.get(tone.lower(), tone_styles["casual"])

        system_message = f"""
        You are an expert dentist and medical writer. Your task is to draft a personalized, patient-friendly follow-up letter based on the provided consultation transcript. 
        Carefully extract and organize relevant details, ensuring completeness and accuracy. Match the letter’s tone to the user's request.
        
        Tone: {selected_tone}
        
        Guidelines:
        - Extract doctor or physician's name and the patient's name and use it in the place holder 
        - If the the name or details of the doctor or patient are missing then omit it or leave the space.
        - If the date is given then use in th eplace holde or else place it as on you last visit 
        - Extract names, dates, symptoms, and observations from the transcript.
        - Summarize findings and treatments concisely but comprehensively.
        - Avoid section titles or headings to match real-world dental letters.
        - Fill in missing details wisely with general advice or omit sections if unavailable.
        - Ensure a smooth, natural flow with clear transitions.
        """

        user_message = f"""
        Dear [Patient Name],

        Thank you for visiting our dental practice on [Date of Appointment]. It was a pleasure meeting you and addressing your dental health concerns.

        Thank you for sharing your symptoms with us. Based on your consultation, we noted the following: 
        Main symptoms: [List main symptoms or omit if not mentioned]. Clinical observations revealed [describe clinical findings, including any inflammation, decay, plaque, or bone loss, or omit if not mentioned]. Relevant X-ray findings include [summarize results or omit if not available].

        Regarding your treatment, we recommend the following procedures: [List recommended treatments, e.g., filling replacement, deep cleaning, root canal, etc., or suggest common preventive measures]. We also suggest maintaining good oral hygiene through [specific hygiene advice or general tips like brushing twice daily and flossing]. Lifestyle adjustments, such as [reducing smoking or modifying diet, or skip if not discussed], may further improve your oral health.

        For your next steps, your scheduled appointment is on [next appointment date, or recommend scheduling one if missing]. In the meantime, we advise [short-term advice, like using desensitizing toothpaste, or provide general comfort measures].

        Please remember that your oral health is our top priority. If you have any questions or experience worsening symptoms, don’t hesitate to contact our office.

        Best wishes for your continued dental health,

        [Physician's Name]
        Physician - Chairsyde 
        ---
        
        Transcript:
        {transcript}
        
        Fill in the structure with extracted information, ensuring coherence and a natural flow.
        """

        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
        )

        return completion.choices[0].message.content

# Example call to API
api_interaction = ApiInteraction()
letter = api_interaction.generate_patient_letter(transcript)
print(letter)  # Display first 500 characters of generated letter


Dear Mr. Thompson,

Thank you for visiting our dental practice on your last visit. It was a pleasure meeting you and discussing your dental health concerns.

During your consultation, you reported experiencing sensitivity in your lower left teeth, particularly when drinking cold water, along with pain while chewing on that side. You mentioned that these symptoms began approximately two to three weeks ago and have become more consistent. Additionally, you noted some bleeding when brushing your teeth, primarily near your molars. 

Upon examination, I observed some plaque buildup along your lower molars and mild gum inflammation. There was also an old composite filling on your lower left first molar, which you indicated was placed about five or six years ago. The probing revealed BP scores of threes and fours, indicating some gum inflammation, along with recession on the lower left side, exposing part of the tooth root. You experienced some discomfort when I tapped on your lower left mola

## Top_g and Temparature changes
### Top_g = 0.9, Temparture= 0.3

In [19]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

client = OpenAI(api_key=os.environ.get('open_api_key'))

temperature = 0.3
max_tokens = 1000
top_p = 0.9

class ApiInteraction:
    def __init__(self):
        self.api_key = os.environ.get('open_api_key')
        if not self.api_key:
            raise ValueError("API key is missing from the environment variables")
        
        # Set the OpenAI API key
        client.api_key = self.api_key

    def generate_patient_letter(self, transcript, tone="empathetic"):
        """
        Generate a detailed and structured patient follow-up letter from a consultation transcript.
        The letter should be accurate, comprehensive, and easy for patients to understand, with tone variation.
        """

        tone_styles = {
            "empathetic": "Use a gentle, caring, and supportive tone, reassuring the patient and expressing empathy for their concerns.",
            "formal": "Maintain a polished, professional tone, focusing on clinical accuracy and straightforward explanations.",
            "casual": "Adopt a friendly, conversational tone to make the patient feel comfortable and at ease.",
        }

        selected_tone = tone_styles.get(tone.lower(), tone_styles["casual"])

        system_message = f"""
        You are an expert dentist and medical writer. Your task is to draft a personalized, patient-friendly follow-up letter based on the provided consultation transcript. 
        Carefully extract and organize relevant details, ensuring completeness and accuracy. Match the letter’s tone to the user's request.
        
        Tone: {selected_tone}
        
        Guidelines:
        - Extract doctor or physician's name and the patient's name and use it in the place holder 
        - If the the name or details of the doctor or patient are missing then omit it or leave the space.
        - If the date is given then use in th eplace holde or else place it as on you last visit 
        - Extract names, dates, symptoms, and observations from the transcript.
        - Summarize findings and treatments concisely but comprehensively.
        - Avoid section titles or headings to match real-world dental letters.
        - Fill in missing details wisely with general advice or omit sections if unavailable.
        - Ensure a smooth, natural flow with clear transitions.
        """

        user_message = f"""
        Dear [Patient Name],

        Thank you for visiting our dental practice on [Date of Appointment]. It was a pleasure meeting you and addressing your dental health concerns.

        Thank you for sharing your symptoms with us. Based on your consultation, we noted the following: 
        Main symptoms: [List main symptoms or omit if not mentioned]. Clinical observations revealed [describe clinical findings, including any inflammation, decay, plaque, or bone loss, or omit if not mentioned]. Relevant X-ray findings include [summarize results or omit if not available].

        Regarding your treatment, we recommend the following procedures: [List recommended treatments, e.g., filling replacement, deep cleaning, root canal, etc., or suggest common preventive measures]. We also suggest maintaining good oral hygiene through [specific hygiene advice or general tips like brushing twice daily and flossing]. Lifestyle adjustments, such as [reducing smoking or modifying diet, or skip if not discussed], may further improve your oral health.

        For your next steps, your scheduled appointment is on [next appointment date, or recommend scheduling one if missing]. In the meantime, we advise [short-term advice, like using desensitizing toothpaste, or provide general comfort measures].

        Please remember that your oral health is our top priority. If you have any questions or experience worsening symptoms, don’t hesitate to contact our office.

        Best wishes for your continued dental health,

        [Physician's Name]
        Physician - Chairsyde 
        ---
        
        Transcript:
        {transcript}
        
        Fill in the structure with extracted information, ensuring coherence and a natural flow.
        """

        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
        )

        return completion.choices[0].message.content

# Example call to API
api_interaction = ApiInteraction()
letter = api_interaction.generate_patient_letter(transcript)
print(letter)  # Display first 500 characters of generated letter


Dear Mr. Thompson,

Thank you for visiting our dental practice on your last visit. It was a pleasure meeting you and discussing your dental health concerns.

I appreciate you sharing your symptoms with us. Based on our consultation, you mentioned experiencing sensitivity in your lower left teeth, particularly when drinking cold water, along with pain while chewing on that side. This discomfort has been consistent for the past two to three weeks. You also noted some bleeding when brushing your teeth, mainly near your molars. 

During the examination, I observed some plaque buildup along your lower molars and mild gum inflammation. Your gum probing scores indicated some inflammation, and there was noticeable recession on your lower left side, exposing part of the tooth root. Additionally, I found an old composite filling on your lower left first molar, which you mentioned was placed about five or six years ago. 

The X-rays revealed mild bone loss around your lower molars and an area of 

### top_g = 0.3 , Temparature = 0.7

In [20]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

client = OpenAI(api_key=os.environ.get('open_api_key'))

temperature = 0.7
max_tokens = 1000
top_p = 0.3

class ApiInteraction:
    def __init__(self):
        self.api_key = os.environ.get('open_api_key')
        if not self.api_key:
            raise ValueError("API key is missing from the environment variables")
        
        # Set the OpenAI API key
        client.api_key = self.api_key

    def generate_patient_letter(self, transcript, tone="empathetic"):
        """
        Generate a detailed and structured patient follow-up letter from a consultation transcript.
        The letter should be accurate, comprehensive, and easy for patients to understand, with tone variation.
        """

        tone_styles = {
            "empathetic": "Use a gentle, caring, and supportive tone, reassuring the patient and expressing empathy for their concerns.",
            "formal": "Maintain a polished, professional tone, focusing on clinical accuracy and straightforward explanations.",
            "casual": "Adopt a friendly, conversational tone to make the patient feel comfortable and at ease.",
        }

        selected_tone = tone_styles.get(tone.lower(), tone_styles["casual"])

        system_message = f"""
        You are an expert dentist and medical writer. Your task is to draft a personalized, patient-friendly follow-up letter based on the provided consultation transcript. 
        Carefully extract and organize relevant details, ensuring completeness and accuracy. Match the letter’s tone to the user's request.
        
        Tone: {selected_tone}
        
        Guidelines:
        - Extract doctor or physician's name and the patient's name and use it in the place holder 
        - If the the name or details of the doctor or patient are missing then omit it or leave the space.
        - If the date is given then use in th eplace holde or else place it as on you last visit 
        - Extract names, dates, symptoms, and observations from the transcript.
        - Summarize findings and treatments concisely but comprehensively.
        - Avoid section titles or headings to match real-world dental letters.
        - Fill in missing details wisely with general advice or omit sections if unavailable.
        - Ensure a smooth, natural flow with clear transitions.
        """

        user_message = f"""
        Dear [Patient Name],

        Thank you for visiting our dental practice on [Date of Appointment]. It was a pleasure meeting you and addressing your dental health concerns.

        Thank you for sharing your symptoms with us. Based on your consultation, we noted the following: 
        Main symptoms: [List main symptoms or omit if not mentioned]. Clinical observations revealed [describe clinical findings, including any inflammation, decay, plaque, or bone loss, or omit if not mentioned]. Relevant X-ray findings include [summarize results or omit if not available].

        Regarding your treatment, we recommend the following procedures: [List recommended treatments, e.g., filling replacement, deep cleaning, root canal, etc., or suggest common preventive measures]. We also suggest maintaining good oral hygiene through [specific hygiene advice or general tips like brushing twice daily and flossing]. Lifestyle adjustments, such as [reducing smoking or modifying diet, or skip if not discussed], may further improve your oral health.

        For your next steps, your scheduled appointment is on [next appointment date, or recommend scheduling one if missing]. In the meantime, we advise [short-term advice, like using desensitizing toothpaste, or provide general comfort measures].

        Please remember that your oral health is our top priority. If you have any questions or experience worsening symptoms, don’t hesitate to contact our office.

        Best wishes for your continued dental health,

        [Physician's Name]
        Physician - Chairsyde 
        ---
        
        Transcript:
        {transcript}
        
        Fill in the structure with extracted information, ensuring coherence and a natural flow.
        """

        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
        )

        return completion.choices[0].message.content

# Example call to API
api_interaction = ApiInteraction()
letter = api_interaction.generate_patient_letter(transcript)
print(letter)  # Display first 500 characters of generated letter


Dear Mr. Thompson,

Thank you for visiting our dental practice on your last visit. It was a pleasure meeting you and discussing your dental health concerns.

I appreciate you sharing your symptoms with us. Based on our consultation, you mentioned experiencing sensitivity in your lower left teeth, particularly when drinking cold water, as well as pain while chewing on that side. You noted that this discomfort began about two to three weeks ago and has become more consistent. Additionally, you reported some bleeding when brushing your teeth, especially near your molars.

During the examination, I observed some plaque buildup along your lower molars and mild gum inflammation. There was also an old composite filling on your lower left first molar, which you mentioned was placed about five or six years ago. The probing revealed some gum inflammation, with BP scores of threes and fours in certain areas, and some recession on your lower left side, exposing part of the tooth root. When I tappe

### top_g =0.5, Temparature = 0.5

In [21]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

client = OpenAI(api_key=os.environ.get('open_api_key'))

temperature = 0.5
max_tokens = 1000
top_p = 0.5

class ApiInteraction:
    def __init__(self):
        self.api_key = os.environ.get('open_api_key')
        if not self.api_key:
            raise ValueError("API key is missing from the environment variables")
        
        # Set the OpenAI API key
        client.api_key = self.api_key

    def generate_patient_letter(self, transcript, tone="empathetic"):
        """
        Generate a detailed and structured patient follow-up letter from a consultation transcript.
        The letter should be accurate, comprehensive, and easy for patients to understand, with tone variation.
        """

        tone_styles = {
            "empathetic": "Use a gentle, caring, and supportive tone, reassuring the patient and expressing empathy for their concerns.",
            "formal": "Maintain a polished, professional tone, focusing on clinical accuracy and straightforward explanations.",
            "casual": "Adopt a friendly, conversational tone to make the patient feel comfortable and at ease.",
        }

        selected_tone = tone_styles.get(tone.lower(), tone_styles["casual"])

        system_message = f"""
        You are an expert dentist and medical writer. Your task is to draft a personalized, patient-friendly follow-up letter based on the provided consultation transcript. 
        Carefully extract and organize relevant details, ensuring completeness and accuracy. Match the letter’s tone to the user's request.
        
        Tone: {selected_tone}
        
        Guidelines:
        - Extract doctor or physician's name and the patient's name and use it in the place holder 
        - If the the name or details of the doctor or patient are missing then omit it or leave the space.
        - If the date is given then use in th eplace holde or else place it as on you last visit 
        - Extract names, dates, symptoms, and observations from the transcript.
        - Summarize findings and treatments concisely but comprehensively.
        - Avoid section titles or headings to match real-world dental letters.
        - Fill in missing details wisely with general advice or omit sections if unavailable.
        - Ensure a smooth, natural flow with clear transitions.
        """

        user_message = f"""
        Dear [Patient Name],

        Thank you for visiting our dental practice on [Date of Appointment]. It was a pleasure meeting you and addressing your dental health concerns.

        Thank you for sharing your symptoms with us. Based on your consultation, we noted the following: 
        Main symptoms: [List main symptoms or omit if not mentioned]. Clinical observations revealed [describe clinical findings, including any inflammation, decay, plaque, or bone loss, or omit if not mentioned]. Relevant X-ray findings include [summarize results or omit if not available].

        Regarding your treatment, we recommend the following procedures: [List recommended treatments, e.g., filling replacement, deep cleaning, root canal, etc., or suggest common preventive measures]. We also suggest maintaining good oral hygiene through [specific hygiene advice or general tips like brushing twice daily and flossing]. Lifestyle adjustments, such as [reducing smoking or modifying diet, or skip if not discussed], may further improve your oral health.

        For your next steps, your scheduled appointment is on [next appointment date, or recommend scheduling one if missing]. In the meantime, we advise [short-term advice, like using desensitizing toothpaste, or provide general comfort measures].

        Please remember that your oral health is our top priority. If you have any questions or experience worsening symptoms, don’t hesitate to contact our office.

        Best wishes for your continued dental health,

        [Physician's Name]
        Physician - Chairsyde 
        ---
        
        Transcript:
        {transcript}
        
        Fill in the structure with extracted information, ensuring coherence and a natural flow.
        """

        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_message}
            ],
            temperature=temperature,
            top_p=top_p,
            max_tokens=max_tokens,
        )

        return completion.choices[0].message.content

# Example call to API
api_interaction = ApiInteraction()
letter = api_interaction.generate_patient_letter(transcript)
print(letter)  # Display first 500 characters of generated letter


Dear Mr. Thompson,

Thank you for visiting our dental practice on your last visit. It was a pleasure meeting you and discussing your dental health concerns.

I appreciate you sharing your symptoms with us. Based on our consultation, you mentioned experiencing sensitivity in your lower left teeth, particularly when drinking cold water, along with pain while chewing on that side. This discomfort has been consistent for the past two to three weeks. You also noted some bleeding when brushing your teeth, especially near your molars, although there was no swelling.

During the examination, I observed some plaque buildup along your lower molars and mild gum inflammation. Additionally, there is an old composite filling on your lower left first molar, which you mentioned was placed about five or six years ago. The probing revealed some gum inflammation with BP scores of threes and fours in certain areas, as well as some recession on the lower left side, exposing part of the tooth root. When I t